In [ ]:
import itertools
import os
from os.path import join
import numpy as np
import pandas as pd

from scipy.spatial.transform import Rotation

from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Plot functions

In [ ]:
def plot_data(x, y, header=""):
    '''Plot data from x and y with subplots.'''
    fig = make_subplots(rows=y.shape[1], cols=1,x_title='Time',)
    
    for index in range(y.shape[1]):
        fig.append_trace(go.Scatter(
            x=x,
            y=y.iloc[:,index],
            name=y.columns[index],
        ), row=index+1, col=1)

    fig.update_layout(height=600, width=600, title_text=header)
    fig.show()

### Other

In [ ]:
def export_timeseries(runs, data_array, folder_name):
    '''Export data from array of data frames 'data_array' to csv file.'''
    export_folder = join('..', 'data', folder_name)
    
    if not os.path.isdir(export_folder):
        os.makedirs(export_folder)
    
    for i, r in enumerate(runs):
        filename = os.path.split(r)
        name = '_'.join(filename[0].split('/')[-2:] + [filename[1]])
        data_array[i].to_csv(join(export_folder, name + '.csv'))

def export_run(run, data, folder_name):
    '''Export data from data frame 'data' to csv file.'''
    export_folder = join('..', 'data', folder_name)
    
    if not os.path.isdir(export_folder):
        os.makedirs(export_folder)
        
    filename = os.path.split(run)
    name = '_'.join(filename[0].split('/')[-2:] + [filename[1]])
    data.to_csv(join(export_folder, name + '.csv'), index=False)

# Data choice

In [ ]:
experiment = 'december'

In [ ]:
fruit = 'orange'

In [ ]:
cut_qualities = ['good'] # good / shallow / deep

In [ ]:
frames = ['ExactoKnife']

# Segmentation options

In [ ]:
desired_nominal_time = 5

# Data extraction

In [ ]:
data_folder = join('..', 'data', 'preprocessed_transformed_data', experiment)
print(data_folder)

In [ ]:
folders  = {cq: join(data_folder, fruit, cq) for cq in cut_qualities}
print(folders)

In [ ]:
all_runs = {cq: [join(folder, file) for file in os.listdir(folder)] for cq, folder in folders.items()}

In [ ]:
opt_position_header = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z') for f in frames))

# Segmentation

In [ ]:
timeseries = {}
for cq, runs in all_runs.items():
    timeseries[cq] = []
    for r in runs:
        print('Processing run ' + r)
        data = pd.read_csv(r)
        
        # cut trajectory based on gradient of force along cutting direction
        gradient = [(b - a) for a, b in
                        zip(data['force_x'][:-1], data['force_x'][1:])]
        max_index = gradient.index(max(gradient))
        min_index = gradient.index(min(gradient))
        ignore_edges_length = -20
        data_segmented = data.loc[max_index + ignore_edges_length:min_index - ignore_edges_length]
        data_segmented.reset_index(inplace=True)
        
        #plot_data(data_segmented['relative_time'], data_segmented[ft_desired_header])
        
        # stretch / shorten cut to desired nominal time
        total_time = data_segmented['relative_time'].iloc[-1] - data_segmented.loc[0, 'relative_time']
        data_segmented.loc[:, 'relative_time'] = (data_segmented['relative_time'] - data_segmented.loc[0, 'relative_time']) / total_time * desired_nominal_time
        
        # set start point of cut to (0,0,0) and calculate displacement along cut
        for column in opt_position_header:
            data_segmented.loc[:, column] -= data_segmented.loc[0, column]
            
        displacement = [np.linalg.norm(x) for x in
                            zip(data_segmented["ExactoKnife_x"], data_segmented["ExactoKnife_y"],
                                data_segmented["ExactoKnife_z"])]
        data_segmented["displacement"] = displacement
        
        export_run(r, data_segmented, join('segmented_data', experiment, fruit, cq))
        timeseries[cq].append(data_segmented)
        
        
        
        

# Segmentation

In [ ]:
fig = make_subplots(rows=3, cols=1,x_title='time [s]')
fig1 = make_subplots(rows=3, cols=1,x_title='time [s]')
fig2 = make_subplots(rows=3, cols=1,x_title='relative displacement from cut start [m]')

timeseries2 = [] 
merge_data2 = pd.DataFrame()

for i, r in enumerate(runs):
    gradient = [(b - a) for a, b in
                        zip(timeseries[i]["force_x"][:-1], timeseries[i]["force_x"][1:])]
    max_index = gradient.index(max(gradient))
    min_index = gradient.index(min(gradient))
    ignore_edges_length = 0
    data_segmented = timeseries[i].loc[max_index + ignore_edges_length:min_index - ignore_edges_length]
    data_segmented.reset_index(inplace=True)
    
    total_time = data_segmented["relative_time"].iloc[-1] - data_segmented.loc[0, "relative_time"]
    data_segmented.loc[:, "relative_time"] = (data_segmented["relative_time"] -
                                                      data_segmented.loc[
                                                          0, "relative_time"]) / total_time * 5
    
    for column in opt_position_header:
        data_segmented.loc[:, column] -= data_segmented.loc[0, column]
    
    #plot_data(data_segmented["relative_time"], data_segmented[ft_desired_header])
    #plot_data(data_segmented["relative_time"], data_segmented[opt_position_header])
    
    angle = np.arctan(data_segmented["ExactoKnife_x"].iloc[-20] / data_segmented["ExactoKnife_y"].iloc[-20])
    R = Rotation.from_euler("z", angle).as_matrix()
    data_segmented.loc[:, "ExactoKnife_x"] = [(R.dot(x))[0] for x in
                                              zip(data_segmented["ExactoKnife_x"],
                                                  data_segmented["ExactoKnife_y"],
                                                  data_segmented["ExactoKnife_z"])]
    data_segmented.loc[:, "ExactoKnife_y"] = [(R.dot(x))[1] for x in
                                              zip(data_segmented["ExactoKnife_x"],
                                                  data_segmented["ExactoKnife_y"],
                                                  data_segmented["ExactoKnife_z"])]
    
    for i, pos in enumerate(opt_position_header):
        fig.append_trace(go.Scatter(
                x=data_segmented["relative_time"],
                y=data_segmented[pos],
                ), row=i+1, col=1)
    for i, force in enumerate(ft_desired_header):
        fig1.append_trace(go.Scatter(
                x=data_segmented["relative_time"],
                y=data_segmented[force],
                ), row=i+1, col=1)
        
    displacement = [np.linalg.norm(x) for x in
                            zip(data_segmented["ExactoKnife_x"], data_segmented["ExactoKnife_y"],
                                data_segmented["ExactoKnife_z"])]
    
    data_segmented["displacement"] = displacement
    timeseries2.append(data_segmented)
    
    for i, force in enumerate(ft_desired_header):
        fig2.append_trace(go.Scatter(
                x=displacement,
                y=data_segmented[force],
                mode='markers',
                showlegend=False,
                ), row=i+1, col=1)
        
    #merge_data2 = pd.concat([merge_data2, data_segmented], axis=0)

#print(merge_data2)
        
export_data(runs, timeseries2, "segmented_data")
#export_data(runs, [merge_data2], "concat_data")

model_x = pd.read_csv(join(data_folder,"model_x.csv"), names=["dis","y","y+","y-"])
model_y = pd.read_csv(join(data_folder,"model_y.csv"), names=["dis","y","y+","y-"])
model_z = pd.read_csv(join(data_folder,"model_z.csv"), names=["dis","y","y+","y-"])



fig.update_yaxes(title_text="position_x [m]", row=1, col=1)
fig.update_yaxes(title_text="position_y [m]", row=2, col=1)
fig.update_yaxes(title_text="position_z [m]", row=3, col=1)
fig.update_layout(height=600, width=800, title_text='-'.join([fruits[0],cut_qualities[0]]),showlegend=False)
fig.show()


fig1.update_yaxes(title_text="force_x [N]", row=1, col=1)
fig1.update_yaxes(title_text="force_y [N]", row=2, col=1)
fig1.update_yaxes(title_text="force_z [N]", row=3, col=1)
fig1.update_layout(height=600, width=800, title_text='-'.join([fruits[0],cut_qualities[0]]),showlegend=False)
fig1.show()

for i, model in enumerate([model_x]):
    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y"],
                    line=dict(color='rgb(0,0,0)', width=3),
                    name="y_predicted",
                    ), row=i+1, col=1)

    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y+"],
                    line=dict(color='rgb(255,0,0)', width=3),
                    name="y_predicted + variance",
                    ), row=i+1, col=1)

    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y-"],
                    line=dict(color='rgb(255,0,0)', width=3),
                    name="y_predicted - variance",
                    ), row=i+1, col=1)

for i, model in enumerate([model_y, model_z]):
    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y"],
                    line=dict(color='rgb(0,0,0)', width=3),
                    showlegend=False,
                    ), row=i+2, col=1)

    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y+"],
                    line=dict(color='rgb(255,0,0)', width=3),
                    showlegend=False,
                    ), row=i+2, col=1)

    fig2.append_trace(go.Scatter(
                    x=model["dis"],
                    y=model["y-"],
                    line=dict(color='rgb(255,0,0)', width=3),
                    showlegend=False,
                    ), row=i+2, col=1)

fig2.update_yaxes(title_text="force_x [N]", row=1, col=1)
fig2.update_yaxes(title_text="force_y [N]", row=2, col=1)
fig2.update_yaxes(title_text="force_z [N]", row=3, col=1)
fig2.update_layout(height=800, width=1000, title_text='-'.join([fruits[0],cut_qualities[0]]))
fig2.show()
    
    

# Export preprocessed data

In [ ]:
if not os.path.isdir(export_folder):
    os.makedirs(export_folder)

In [ ]:
for i, r in enumerate(runs):
    filename = os.path.split(r)
    name = '_'.join(filename[0].split('/')[-2:] + [filename[1]])
    timeseries[i].to_csv(join(export_folder, name + '.csv'))